In [64]:
from deepuq.models.models import model_setup_DER
from deepuq.data.data import DataPreparation
from deepuq.train import train
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

In [65]:
# necessary in order to display matplotlib plots in a notebook environment
# when also running python scripts inline
%matplotlib inline

In [66]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss_type = 'DER'
model, lossFn = model_setup_DER(loss_type, DEVICE)

In [67]:
uniform = True
norm = False
verbose = False
val_prop = 0.1
rs_prior = 42
rs_uniform = 42
rs_train_val = 42
BATCH_SIZE = 100
lr = 0.001
COEFF = 0.01
n_epochs = 100
out_dir = '../DeepUQResources/'

In [68]:
experiments_df = {
    "0D, input, low": {
        "size_df": 100000,
        "noise": "low",
        "dim": "0D",
        "injection": "input"
    },
    "0D, input, medium": {
        "size_df": 100000,
        "noise": "medium",
        "dim": "0D",
        "injection": "input"
    },
    "0D, input, high": {
        "size_df": 100000,
        "noise": "high",
        "dim": "0D",
        "injection": "input"
    },
    "0D, output, low": {
        "size_df": 100000,
        "noise": "low",
        "dim": "0D",
        "injection": "output"
    },
    "0D, output, medium": {
        "size_df": 100000,
        "noise": "medium",
        "dim": "0D",
        "injection": "output"
    },
    "0D, output, high": {
        "size_df": 100000,
        "noise": "high",
        "dim": "0D",
        "injection": "output"
    },
    "2D, input, low": {
        "size_df": 5000,
        "noise": "low",
        "dim": "2D",
        "injection": "input"
    },
    "2D, input, medium": {
        "size_df": 5000,
        "noise": "medium",
        "dim": "2D",
        "injection": "input"
    },
    "2D, input, high": {
        "size_df": 5000,
        "noise": "high",
        "dim": "2D",
        "injection": "input"
    },
    "2D, output, low": {
        "size_df": 5000,
        "noise": "low",
        "dim": "2D",
        "injection": "output"
    },
    "2D, output, medium": {
        "size_df": 5000,
        "noise": "medium",
        "dim": "2D",
        "injection": "output"
    },
    "2D, output, high": {
        "size_df": 5000,
        "noise": "high",
        "dim": "2D",
        "injection": "output"
    },
}

In [69]:
#counter = 0
for experiment_name, params in experiments_df.items():
    print(f"Experiment: {experiment_name}")
    #for key, value in params.items():
    #    print(f"Key, value pairs - {key}: {value}")
    data = DataPreparation()
    model_inputs, model_outputs = data.generate_df(
                params["size_df"], params["noise"],
                params["dim"], params["injection"], uniform, verbose,
                rs_prior=rs_prior, rs_uniform=rs_uniform)
    model_inputs, model_outputs, norm_params = data.normalize(
        model_inputs, model_outputs, norm
    )
    x_train, x_val, y_train, y_val = data.train_val_split(
        model_inputs,
        model_outputs,
        val_proportion=val_prop,
        random_state=rs_train_val,
    )
    trainData = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
    trainDataLoader = DataLoader(
        trainData, batch_size=BATCH_SIZE, shuffle=False
    )
    train.train_DER(
            trainDataLoader,
            x_val,
            y_val,
            lr,
            DEVICE,
            COEFF,
            'DER',
            norm_params,
            model_name='DER',
            EPOCHS=n_epochs,
            path_to_model=out_dir,
            inject_type=params["injection"],
            data_dim=params["dim"],
            noise_level=params["noise"],
            save_all_checkpoints=True,
            save_final_checkpoint=True,
            overwrite_final_checkpoint=True,
            plot=False,
            savefig=False,
            set_and_save_rs=True,
            rs=2
        )

Experiment: 0D, input, low
number of sims 500000
YES WERE VARYING SIGMA
0D data generated,                 with noise injected type: input.
[INFO] training the network...
saving all checkpoints?
True
saving final checkpoint?
True
overwriting final checkpoint if its already there?
True
saving here: ../DeepUQResources/
model name: DER
setting and saving the rs
model is Sequential(
  (0): Model(
    (model): Sequential(
      (0): Linear(in_features=3, out_features=64, bias=True)
      (1): ReLU()
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ReLU()
      (4): Linear(in_features=64, out_features=4, bias=True)
    )
  )
  (1): DERLayer()
) lossfn <function loss_der at 0x11d598820>
epoch 0 0.0
new best loss -2.3553004264831543 in epoch 0
meanwhile mse is 0.0007639101822860539
epoch 1 0.01
new best loss -2.4778695106506348 in epoch 1
meanwhile mse is 0.00033444457221776247
epoch 2 0.02
new best loss -2.659911632537842 in epoch 2
meanwhile mse is 0.00024884997401386

In [70]:
print('done training')

done training
